In [1]:
from clean2 import *

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
df, temp = data_wrangling()

In [ ]:
df = data_wrangling()
tmp = df.groupby(
        ["C/A", "UNIT", "SCP", "STATION", "DATE", "AMPM", "DAY_NAME"], as_index=False
    ).ENTRIES.max()

##
t = df.groupby(
    ["C/A", "UNIT", "SCP", "STATION", "DATE", "AMPM", "DAY_NAME"], as_index=False
).EXITS.max()
tmp["EXITS"] = t["EXITS"]

# create prev_date and prev_entries cols by shifting these columns forward one day
# if shifting date and entries, don't group by date
tmp[["PREV_DATE", "PREV_ENTRIES", "PREV_EXITS"]] = tmp.groupby(
    ["C/A", "UNIT", "SCP", "STATION", "AMPM"]
)[["DATE", "ENTRIES", "EXITS"]].apply(lambda grp: grp.shift(1))

# Drop the rows for the earliest date in the df, which are now NaNs for prev_date and prev_entries cols
tmp.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

tmp = get_tmp_entries(df)

In [ ]:
df.sample(10, random_state=10)

In [ ]:
tmp

In [ ]:
tmp['MEAN_TRAFFIC'] = round(tmp.TMP_ENTRIES + tmp.TMP_EXITS, 0)
AMPM = ['AM', 'PM']

# TOP TEN AM
mask = tmp.AMPM == 'AM'
tmp[mask].groupby('STATION').agg('mean').sort_values(['traffic']).head(10)

In [ ]:
# TOP TEN AM
mask = tmp.AMPM == 'AM'
am_df = tmp[mask].groupby('STATION').agg('mean').sort_values(['MEAN_TRAFFIC'], ascending=False).head(11)
am_df.reset_index()[['STATION', 'MEAN_TRAFFIC']].shift(1).dropna(how='all')

In [ ]:
# TOP TEN PM
mask = tmp.AMPM == 'PM'
am_df = tmp[mask].groupby('STATION').agg('mean').sort_values(['MEAN_TRAFFIC'], ascending=False).head(11)
am_df.reset_index()[['STATION', 'MEAN_TRAFFIC']].shift(1).dropna(how='all')